In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Primero guardamos las coordenadas del centro de masa para cada tiempo

In [ ]:
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph
import numpy as np

vector = ('00','01','03')
vector2 = ('M31', 'MW', 'M33')
h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter

# path = '/home/ornela/SimCLUES/'
path = '/home/omarioni/'
atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]

path = '/mnt/is2/alejandro/ornella/'

snapshot=range(496,495,-1)

for isnap in snapshot:
    snap = h5py.File(path + 'outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')
    
    for i in range(0,3):
        cm   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Center'].value
        r200 = snap['subhalo_0'+ str('%s'%vector[i])+ '/R200'].value
        
        pstr  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Coordinates'].value
        mstr  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Masses'].value
        vel   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/Velocities'].value
        IDs   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/Ids'].value
        tform = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/FormationTime'].value
            
        z = a0/aexp[0] - 1.
        Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
        
#---aca paso las coordenadas respecto al centro de la galaxia------
        xstr = (pstr[:,0]-cm[0])*aexp[0]/h
        ystr = (pstr[:,1]-cm[1])*aexp[0]/h
        zstr = (pstr[:,2]-cm[2])*aexp[0]/h
        rstr = np.sqrt(xstr**2+ystr**2+zstr**2)
        
        r200 = r200*aexp[0]/h
        
        rgal=0.15*r200
        
        limit, = np.where(rstr<rgal)
        
        x

#----------------------------------------------------------------------------------

        data_tot = np.zeros([np.size(rtot_sort),3])
        data_tot[:,0]= rtot_sort
        data_tot[:,1]= Vtot
        data_tot[:,2]= Mc_tot*1e10
        
        data_str = np.zeros([np.size(rstr_sort),3])
        data_str[:,0]= rstr_sort
        data_str[:,1]= Vstr
        data_str[:,2]= Mc_str*1e10
        
        data_gas = np.zeros([np.size(rgas_sort),3])
        data_gas[:,0]= rgas_sort
        data_gas[:,1]= Vgas
        data_gas[:,2]= Mc_gas*1e10
        
        data_drk = np.zeros([np.size(rdrk_sort),3])
        data_drk[:,0]= rdrk_sort
        data_drk[:,1]= Vdrk
        data_drk[:,2]= Mc_drk*1e10
        
        data = np.ndarray([5,1])
        data[0] = r200
        data[1] = r50
        data[2] = r25
        data[3] = r75
        data[4] = M_gal*1e10

        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vtot.dat', data_tot)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vstr.dat', data_str)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vgas.dat', data_gas)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vdrk.dat', data_drk)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_radios.dat', data)

In [ ]:
import matplotlib.pyplot as plt
import h5py
import rotation as rot
import sphviewer as sph
import numpy as np

vector = ('00','01','03')
vector2 = ('M31', 'MW', 'M33')
h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter

# path = '/home/ornela/SimCLUES/'
path = '/home/omarioni/'
atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]

path = '/mnt/is2/alejandro/ornella/'

snapshot=range(496,495,-1)

for isnap in snapshot:
    snap = h5py.File(path + 'outputs_1/snap_'+str('%03d'%isnap)+'.h5py', 'r')
    
    for i in range(0,3):
        cm   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Center'].value
        r200 = snap['subhalo_0'+ str('%s'%vector[i])+ '/R200'].value
        
        pstr  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Coordinates'].value
        mstr  = snap['subhalo_0'+ str('%s'%vector[i]) + '/Str/Masses'].value
        vel   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/Velocities'].value
        IDs   = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/Ids'].value
        tform = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/FormationTime'].value
            
        z = a0/aexp[0] - 1.
        Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
        
#---aca paso las coordenadas respecto al centro de la galaxia------
        xstr = (pstr[:,0]-cm[0])*aexp[0]/h
        ystr = (pstr[:,1]-cm[1])*aexp[0]/h
        zstr = (pstr[:,2]-cm[2])*aexp[0]/h
        rstr = np.sqrt(xstr**2+ystr**2+zstr**2)
        
        r200 = r200*aexp[0]/h
        
        rgal=0.15*r200
        
        limit, = np.where(rstr<rgal)
        
        x

#----------------------------------------------------------------------------------

        data_tot = np.zeros([np.size(rtot_sort),3])
        data_tot[:,0]= rtot_sort
        data_tot[:,1]= Vtot
        data_tot[:,2]= Mc_tot*1e10
        
        data_str = np.zeros([np.size(rstr_sort),3])
        data_str[:,0]= rstr_sort
        data_str[:,1]= Vstr
        data_str[:,2]= Mc_str*1e10
        
        data_gas = np.zeros([np.size(rgas_sort),3])
        data_gas[:,0]= rgas_sort
        data_gas[:,1]= Vgas
        data_gas[:,2]= Mc_gas*1e10
        
        data_drk = np.zeros([np.size(rdrk_sort),3])
        data_drk[:,0]= rdrk_sort
        data_drk[:,1]= Vdrk
        data_drk[:,2]= Mc_drk*1e10
        
        data = np.ndarray([5,1])
        data[0] = r200
        data[1] = r50
        data[2] = r25
        data[3] = r75
        data[4] = M_gal*1e10

        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vtot.dat', data_tot)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vstr.dat', data_str)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vgas.dat', data_gas)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_Vdrk.dat', data_drk)
        np.savetxt('/home/omarioni/Barras_GdGs/Barras_Gd/_data/'+str('%s'%vector2[i])+'_radios.dat', data)